In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,10.05,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,...,0.0,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.0,0.0,-0.0
3,-0.00,7.78,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,...,-0.0,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.0,-0.0,-0.0
4,0.00,0.00,6.77,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,...,0.0,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
5,-0.00,0.00,0.00,5.44,0.00,-0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,0.0
6,-0.00,0.00,-0.00,0.00,4.51,0.00,0.00,-0.00,-0.00,-0.00,...,-0.0,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
7,0.00,0.00,-0.00,-0.00,0.00,3.62,0.00,-0.00,0.00,0.00,...,-0.0,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.0,-0.0,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,3.02,0.00,0.00,-0.00,...,-0.0,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.0,-0.0,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,2.52,-0.00,0.00,...,0.0,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.0,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,2.09,0.00,...,0.0,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,1.74,...,0.0,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00004,0.00003,-0.00004,0.00003,-0.00000,...,0.00004,0.00003,0.00004,-0.00005,-0.00000,-0.00000,0.00006,0.00003,0.00007,-0.00002
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00004,-0.00000,...,-0.00003,-0.00004,-0.00004,-0.00002,-0.00002,-0.00000,0.00001,0.00003,-0.00002,-0.00006
4,0.00002,0.00000,1.00000,0.00007,-0.00003,-0.00001,0.00002,-0.00001,0.00002,0.00000,...,0.00001,-0.00001,-0.00001,-0.00003,0.00003,-0.00003,-0.00000,0.00005,0.00001,0.00005
5,-0.00003,0.00002,0.00007,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00002,0.00003,-0.00001,-0.00003,-0.00004,0.00004,-0.00000,0.00004,-0.00001,0.00001
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00003,0.00002,-0.00001,-0.00000,0.00002,0.00002,0.00001
7,0.00004,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00006,-0.00001,-0.00000,-0.00001,0.00004,0.00002,-0.00007,-0.00005,-0.00003,0.00008
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,-0.00000,...,-0.00005,0.00000,0.00002,0.00000,-0.00007,-0.00002,-0.00000,0.00001,-0.00004,-0.00003
9,-0.00004,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00001,-0.00007,-0.00001,-0.00001,-0.00004,-0.00002,-0.00001,0.00001,0.00003,-0.00000
10,0.00003,0.00004,0.00002,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00001,-0.00001,0.00001,-0.00006,0.00002,-0.00001,0.00002,-0.00001,0.00001,0.00001
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00001,0.00005,1.00000,...,0.00002,-0.00003,0.00001,-0.00001,-0.00003,0.00002,-0.00005,0.00001,-0.00003,-0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.001751235213339885

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.16545257e+00 1.89912012e+00 1.43096884e+00 9.23096240e-01
 6.33379560e-01 4.09030702e-01 2.83230865e-01 1.97515411e-01
 1.35349161e-01 9.36364855e-02 6.33486820e-02 4.68118434e-02
 2.86684853e-02 2.05848026e-02 1.37181390e-02 9.02689465e-03
 5.80227627e-03 3.71250865e-03 2.31656569e-03 1.49106147e-03
 8.82642294e-04 5.11835628e-04 2.91141086e-04 1.65442482e-04
 8.57527262e-05 4.26330027e-05 2.07850952e-05 9.31893775e-06
 1.88879964e-06 4.76915631e-07 9.34173754e-08 6.22797726e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997034,-0.058504,-0.036354,-0.022342,-0.016317,-0.011947,-0.009500,-0.007750,-0.006249,-0.005144,...,-0.000281,-0.000214,-0.000155,-0.000115,-0.000079,-0.000055,-0.000023,-0.000013,-0.000005,-0.000003
1,0.052118,0.990676,-0.113095,-0.040311,-0.025373,-0.017143,-0.013205,-0.010475,-0.008307,-0.006775,...,-0.000372,-0.000283,-0.000206,-0.000145,-0.000103,-0.000071,-0.000032,-0.000016,-0.000008,-0.000005
2,0.038942,0.105362,0.988737,-0.080474,-0.041292,-0.025484,-0.018784,-0.014600,-0.011458,-0.009246,...,-0.000493,-0.000377,-0.000274,-0.000196,-0.000135,-0.000097,-0.000043,-0.000022,-0.000011,-0.000005
3,0.024451,0.042744,0.068702,0.990078,-0.095571,-0.041187,-0.027169,-0.019868,-0.015138,-0.011953,...,-0.000609,-0.000462,-0.000339,-0.000243,-0.000172,-0.000117,-0.000053,-0.000027,-0.000014,-0.000007
4,0.019209,0.029448,0.040115,0.084666,0.989175,-0.086551,-0.044457,-0.029545,-0.021279,-0.016271,...,-0.000788,-0.000598,-0.000436,-0.000311,-0.000218,-0.000153,-0.000069,-0.000036,-0.000017,-0.000009
5,0.014136,0.020145,0.025410,0.040091,0.073242,0.987913,-0.105204,-0.049789,-0.031293,-0.022443,...,-0.000989,-0.000745,-0.000541,-0.000385,-0.000268,-0.000188,-0.000089,-0.000047,-0.000022,-0.000009
6,0.011835,0.016365,0.019852,0.028434,0.042383,0.090445,0.985094,-0.113478,-0.051464,-0.032954,...,-0.001250,-0.000939,-0.000680,-0.000484,-0.000349,-0.000241,-0.000108,-0.000057,-0.000029,-0.000014
7,0.010177,0.013726,0.016338,0.022153,0.030411,0.049530,0.096387,0.983642,-0.114747,-0.053909,...,-0.001565,-0.001201,-0.000863,-0.000612,-0.000438,-0.000305,-0.000137,-0.000072,-0.000034,-0.000017
8,0.008508,0.011306,0.013327,0.017574,0.022955,0.033260,0.050255,0.095701,0.982394,-0.122208,...,-0.001958,-0.001488,-0.001076,-0.000773,-0.000539,-0.000380,-0.000169,-0.000091,-0.000043,-0.000021
9,0.007351,0.009683,0.011307,0.014624,0.018562,0.025403,0.034851,0.052247,0.101936,0.981107,...,-0.002473,-0.001887,-0.001359,-0.000970,-0.000688,-0.000477,-0.000218,-0.000114,-0.000056,-0.000027


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.002965808667178882,
 0.009323579008914562,
 0.011262801063633088,
 0.009922387282507827,
 0.010825281109281604,
 0.012087170131419245,
 0.01490639956471751,
 0.01635830916017378,
 0.017605649358655606,
 0.018893315594467053,
 0.024167438007689723,
 0.024763639466491738,
 0.023436589954904385,
 0.02844115363630284,
 0.02693674681169611,
 0.027557042062681503,
 0.028836390798759992,
 0.030732510407773916,
 0.03482711479101941,
 0.03750998778196413,
 0.03620256065133476,
 0.038925779661348914,
 0.04441506606894252,
 0.0484205074763292,
 0.049918707824064934,
 0.058210708104068476,
 0.07312173909527364,
 0.06385042226178661,
 0.04581426683070011,
 0.07319193643007138,
 0.15072263707607458,
 0.13126981646629043]